# Discussion Questions

### Introduction

In this lesson, we'll work through navigating the codebase.

### Navigating the codebase

In navigating the codebase, it's good to start broadly and move narrowly.

**1. Airflow**

The first for seeing the entire flow is to go to airflow.  There, we can see the extract load, transform_load and load data_to_db tasks.  

Notice that each of the tasks are relatively small -- essentially, they just call another function and provide some logging.  

a) What are the benefits of keeping our codebase isolated from airflow.

b) Are there downsides to having so much of our code outside of airflow?

c) Why did we split our code into three different tasks?  Could we have just used a single task -- for example perform ELTL in one step? 

**2. Lambda**

* What are the benefits to hosting much of our code on lambda functions?  Are there downsides?

**3. What about dockerizing our code?**
* What are the benefits

**4. Architecture**

The architecture we chose is one of extract-load and then transform-load.  

a) What is the benefit to storing both raw and transformed data? 

b) Should we have left the transformation until after the data is loaded in the database (and transform with dbt/sql?).  Any benefits you could see to doing so?  Any downsides?  (It's ok if you're not quite sure, we'll talk more about this).  

**5. Code navigation**

In navigating the solution codebase, and looking at the `extract_load` service or the `transform_load` service -- which file should we start at?  (A one word answer is fine).

# Answers

a) By keeping isolated from airflow, we can more easily:

1. Test our functions (we can do this outside of airflow, and isolate how our functions work outside of airflow by just calling them directly)
2. Move to a different orchestrator instead of airflow.  Or trigger our functions from different events inside of lambda.
3. Keep airflow simple -- it's already difficult to maintain from a devops perspective.

b) Downsides

1. Increases the level of abstraction -- we now have our code abstracted to lambda functions.  This means that our code is *hidden* away in different lambda functions -- it's a little more indirect.  This probably is overweighed by the benefits.

2. The biggest downside is that the logging between airflow and lambda functions is not very smooth.  For example, when there is an error in lambda, we do not currently see that error message in airflow, and same thing with the return value from our lambda function.  Ideally, the return value would be passed onto airflow. 

c) Multiple tasks

* With multiple tasks, we can have airflow automatically retry the task if a task breaks.  
* By keeping the tasks small, we also get a level of predictability as to the output of each task.  Given an input we get an output.  So we can think of each task like functions (which they are).  We provide an input and get an output.  So for example, with the EL portion we provide parameters and get the output of the raw response (which we store in s3).  With the transform step we provide the raw data and get the output of the transformed data.

* Similarly to functions (whether Python functions, or lambda functions) a good metric is do they add user value -- if they do, we can keep it as a separate task.  And then we have the option of choosing our which features we want.

* So for example, even with the loading to postgres task -- it's probably best to keep this separate -- as we may or may not want our data stored in a running database (or may want to choose the database we load to).  

2. Lambda functions

* With lambda functions, we can have our code hosted and deployed with a pay as you go cost model, and can let the lambda service take care of a lot of the devops responsibilities.  The same applies to this application.  We minimize what airflow is in charge of, and have a lot of the costs hosted there.

* Coordination with other events.  With lambda functions we can also have them triggered by other events in the AWS ecosystem -- or can invoke our lambdas from boto to integrate with other codebases.

3. Dockerizing

Once again, we get increased flexibility.  Each one of our tasks is essentially a *service*.  If we want Airflow to boot up and run these docker containers we can, or we can continue to deploy them to lambda.  With docker we have specified the environmental dependencies of running each of the tasks.

4. Architecture

A) Raw and Transformed

* Raw data - Avoid data loss.  By storing the raw data we make sure we are not losing any of our data.  We can always retrieve it later and transform it later.  Raw data may be beneficial to ML engineers/data scientists who are looking to extract multiple features from the data.

* Transformed data - With transformed data, we make it easier to load our data to a database and query our data.  This is useful for less technical individuals, and to ultimately load into a data warehouse.

In other words, above we employ our lakehouse structure.  We store the raw data for a data lake, and the transformed data for a data warehouse.

B) Transformation Layer 

* By storing our raw data in the database, and transforming it from there we could:

1. Make our data more accessible, as it's in the database and so easier to query
2. Potentially make it easier to track the data lineage -- as we can see how our data goes from the raw data to the transformed data all through the database

* Downsides
    * It's typically easier to flatten our data in Python/pandas than in SQL.  However, databases like Snowflake currently allow us to work with pandas through [snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes)
* If not using snowflake, perhaps a good middle ground is just to flatten the data using the [json normalize function](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html) in pandas, and then select and transform the data in SQL from there.

By doing this, we are strictly keeping airflow for the ingestion layer, and using a tool like DBT for the transformation layer.

5. Code navigation

* We can start at the `main.py` file.  This is similar to the routes in a flask application.  It kicks off the execution of our codebase, and ideally shows us the main functions that are called from there.